<a href="https://colab.research.google.com/github/rvssridatta/ML/blob/main/22_MovieRecommedSVD/22_MovieRecommedSVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd


In [23]:
#importing and parsing the dataset as ratings and movies details

ratingData = pd.io.parsers.read_csv('ratings.dat', names = ['user_id','movie_id','rating','time'],
                                    engine = 'python',delimiter = '::')
movieData = pd.io.parsers.read_csv('movies.dat',names = ['movie_id','title','genre'],engine = 'python',delimiter ='::',encoding='latin-1')
#print(movieData)

In [24]:
#rating matrix creation
ratingMatrix = np.ndarray(shape = (np.max(ratingData.movie_id.values),np.max(ratingData.user_id.values)),
                          dtype = np.uint8)
ratingMatrix[ratingData.movie_id.values -1,ratingData.user_id.values-1] = ratingData.rating.values
print(ratingMatrix)

[[ 5  0  0 ...  0  0  3]
 [ 9  0  0 ...  0  0  0]
 [15  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [25]:
#normalization SUBTRACT MEANOFF - NORMALIZATION

normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix,1))]).T
print(normalizedMatrix)

[[ 3.16804636 -1.83195364 -1.83195364 ... -1.83195364 -1.83195364
   1.16804636]
 [ 7.39884106 -1.60115894 -1.60115894 ... -1.60115894 -1.60115894
  -1.60115894]
 [12.82301325 -2.17698675 -2.17698675 ... -2.17698675 -2.17698675
  -2.17698675]
 ...
 [-0.03278146 -0.03278146 -0.03278146 ... -0.03278146 -0.03278146
  -0.03278146]
 [-0.02582781 -0.02582781 -0.02582781 ... -0.02582781 -0.02582781
  -0.02582781]
 [-0.24288079 -0.24288079 -0.24288079 ... -0.24288079 -0.24288079
  -0.24288079]]


In [26]:
#computing SVD

A = normalizedMatrix.T/np.sqrt(ratingMatrix.shape[0]-1)
U,S,V = np.linalg.svd(A)

In [32]:
#calculate cosine similarity,sort by most similar and return the top N
def similar(ratingData,movie_id,top_n = 10):
  index  = movie_id -1 #movie id starts from 1
  movie_row = ratingData[index, :]
  magnitude = np.sqrt(np.einsum('ij,ij->i',ratingData,ratingData)) #einstein summation |traditional matrix multiplication and is equivalent to np.matmul(a,b)
  similarity = np.dot(movie_row,ratingData.T)/(magnitude[index]* magnitude)
  sort_indexes = np.argsort(-similarity)#perform an indirect sort along the given axis (last axis)
  return sort_indexes[:top_n]

In [35]:
#select k principal components and a movie_id to represent recomendedations and print top_n list

k = 50
movie_id = 2
top_n = 5

sliced = V.T[:, :k] #representative data
indexes = similar(sliced,movie_id,top_n)

print('Recommendations for movie {0}: \n'.format(
    movieData[movieData.movie_id == movie_id].title.values[0]))
for id in indexes + 1:
  print(movieData[movieData.movie_id == id].title.values[0])


Recommendations for movie Jumanji (1995): 

Jumanji (1995)
Toy Story (1995)
Happy Gilmore (1996)
Tombstone (1993)
Grumpier Old Men (1995)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  
